# Dataset Caching

- Copying the mitosis dataset to Kaggle's working directory to speed up data access.
- Checks if a cached copy exists to avoid redundant copying.
- Sets `dataset_dir` for consistent usage in later cells.


In [1]:
import os, shutil

# Define source and destination
src = "/kaggle/input/mitosis-detection-dataset"
dst = "/kaggle/working/mitosis-dataset-cache"

# Copy only if not already cached
if not os.path.exists(dst):
    print("📦 Caching dataset for faster access...")
    shutil.copytree(src, dst)
else:
    print("✅ Using cached dataset!")

# Find all images recursively
image_files = []
for root, _, files in os.walk(dst):
    for f in files:
        if f.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_files.append(os.path.join(root, f))

print(f"📁 Dataset directory set to: {dst}")
print(f"🖼️ Total images found: {len(image_files)}")


📦 Caching dataset for faster access...
📁 Dataset directory set to: /kaggle/working/mitosis-dataset-cache
🖼️ Total images found: 43416


# Libraries and Modules

- PyTorch & torchvision for deep learning and Faster R-CNN.
- PIL and NumPy for image handling.
- `autocast` and `GradScaler` for mixed precision training.
- `tqdm` for progress bars.


In [2]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler

import torchvision
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.ndimage import binary_opening, binary_closing, label


!pip install -q segmentation-models-pytorch


seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

print(" Environment ready with fixed seed for reproducibility.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.8 MB/s eta 0:00:00:00:0100:01
 Environment ready 

# Dataset Exploration

- Iterates through directories to inspect dataset structure.
- Displays the first few files in each folder to verify images and labels.


In [3]:

for dirname, _, filenames in os.walk(dst):  
    print(f"\n📁 Directory: {dirname}")
    for i, filename in enumerate(filenames[:10]):  # Show only first 10 files
        print("   ", os.path.join(dirname, filename))
    if len(filenames) > 10:
        print(f"   ... and {len(filenames)-10} more files")


📁 Directory: /kaggle/working/mitosis-dataset-cache
    /kaggle/working/mitosis-dataset-cache/README.dataset.txt
    /kaggle/working/mitosis-dataset-cache/README.roboflow.txt
    /kaggle/working/mitosis-dataset-cache/data.yaml

📁 Directory: /kaggle/working/mitosis-dataset-cache/test

📁 Directory: /kaggle/working/mitosis-dataset-cache/test/images
    /kaggle/working/mitosis-dataset-cache/test/images/A12_01Dd_jpg.rf.942266cea7b2ec998278e865b6b8ad24.jpg
    /kaggle/working/mitosis-dataset-cache/test/images/A11_07Aa_jpg.rf.f2a9192e0a5b2e1af42f6acc05d9393e.jpg
    /kaggle/working/mitosis-dataset-cache/test/images/A10_04Ac_jpg.rf.2abe52fbfb07bb78079ef5080fc1ec04.jpg
    /kaggle/working/mitosis-dataset-cache/test/images/A10_00Ac_jpg.rf.3ee13137a67d9deddaa90e34ab444ba8.jpg
    /kaggle/working/mitosis-dataset-cache/test/images/A12_03Cc_jpg.rf.2c65bb12ceb498869e85453d9a6488f6.jpg
    /kaggle/working/mitosis-dataset-cache/test/images/A11_04Cd_jpg.rf.345886b73294bfcd034f15d244291d23.jpg
    /kaggl

#  Positive and Negative Image Split

- Classifies training images into positive (mitosis present) and negative (mitosis absent).
- Counts total positive and negative images for dataset balancing.


In [4]:

train_labels_dir = os.path.join(dst, "train/labels")
train_images_dir = os.path.join(dst, "train/images")

positive_images = []  # Images that contain mitosis (at least one label)
negative_images = []  # Images without mitosis

for label_file in os.listdir(train_labels_dir):
    label_path = os.path.join(train_labels_dir, label_file)
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # If label file has bounding boxes, it's positive
    if len(lines) > 0:
        positive_images.append(label_file.replace('.txt', '.jpg'))
    else:
        negative_images.append(label_file.replace('.txt', '.jpg'))

print(f"✅ Total positive images: {len(positive_images)}")
print(f"✅ Total negative images: {len(negative_images)}")


✅ Total positive images: 10765
✅ Total negative images: 15299


# Custom Dataset Class

- Loads images and labels for Faster R-CNN training.
- Converts YOLO-format labels into bounding boxes.
- Handles cases with no objects by creating empty tensors.
- Applies transformations and normalization to images.


In [5]:
class MitosisDataset(Dataset):
    def __init__(self, images_dir, labels_dir, subset_images=None, transforms=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transforms = transforms

        if subset_images is not None:
            self.image_files = subset_images
        else:
            self.image_files = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        img = Image.open(img_path).convert("RGB")
        img = np.array(img)

        # Resize and transform
        if self.transforms:
            img = self.transforms(Image.fromarray(img))
        else:
            img = torch.tensor(img, dtype=torch.float32).permute(2,0,1) / 255.0

        # Load labels
        label_file = img_name.replace('.jpg', '.txt')
        label_path = os.path.join(self.labels_dir, label_file)
        boxes, labels = [], []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) > 0:
                        x_center, y_center, w, h = map(float, parts[1:])
                        img_w, img_h = img.shape[2], img.shape[1]
                        xmin = (x_center - w/2) * img_w
                        ymin = (y_center - h/2) * img_h
                        xmax = (x_center + w/2) * img_w
                        ymax = (y_center + h/2) * img_h
                        # Skip degenerate boxes
                        if xmax > xmin and ymax > ymin:
                            boxes.append([xmin, ymin, xmax, ymax])
                            labels.append(1)

        if len(boxes) == 0:
            boxes = torch.zeros((0,4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}
        return img, target

# Subset Selection

- Chooses a balanced subset of positive and negative images for training.
- Ensures randomness and shuffles for better generalization.


In [6]:
num_pos = 10765  # total positive samples
num_neg = 5382   # total negative samples

# Randomly sample a balanced subset (~7000 images total)
subset_pos = random.sample(positive_images, 4667)  # about 2/3 positive
subset_neg = random.sample(negative_images, 2333)  # about 1/3 negative

# Combine and shuffle
subset_images = subset_pos + subset_neg
random.shuffle(subset_images)

print(f"✅ Selected subset: {len(subset_images)} images ({len(subset_pos)} positive, {len(subset_neg)} negative)")


✅ Selected subset: 7000 images (4667 positive, 2333 negative)


# Image Transformations

- Resizes all images to 512x512 for Faster R-CNN.
- Converts images to tensors and scales pixel values to [0,1].


In [7]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((512,512)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.ToTensor()
])


print("✅ Transformations ready: Resize → ToTensor")


✅ Transformations ready: Resize → ToTensor


#  Model Setup

- Loads Faster R-CNN with ResNet-50 backbone pre-trained on COCO.
- Adjusts the classifier for 5 classes (mitosis + background).
- Moves model to GPU if available.


In [8]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"🚀 Model ready on {device}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 178MB/s]  


🚀 Model ready on cuda


# DataLoader Setup

- Wraps datasets in PyTorch DataLoaders for batching.
- Uses custom `collate_fn` for object detection targets.
- Shuffle training data; keep validation data in order.


In [9]:
train_dataset = MitosisDataset(
    images_dir="/kaggle/working/mitosis-dataset-cache/train/images",
    labels_dir="/kaggle/working/mitosis-dataset-cache/train/labels",
    subset_images=subset_images,
    transforms=transform
)

valid_dataset = MitosisDataset(
    images_dir="/kaggle/working/mitosis-dataset-cache/valid/images",
    labels_dir="/kaggle/working/mitosis-dataset-cache/valid/labels",
    transforms=transform
)

print("✅ Datasets created successfully!")


✅ Datasets created successfully!


# Optimizer and Scheduler

- Stochastic Gradient Descent (SGD) with momentum and weight decay.
- StepLR reduces learning rate every 3 epochs to improve convergence.


In [10]:
import os

train_img_dir = "/kaggle/working/mitosis-dataset-cache/train/images"
missing_files = [img for img in subset_images if not os.path.exists(os.path.join(train_img_dir, img))]

print(f"❌ Missing {len(missing_files)} files.")
if len(missing_files) > 0:
    print("Example missing files:", missing_files[:10])


❌ Missing 0 files.


In [11]:
train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=lambda x: tuple(zip(*x))
)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=2,
    shuffle=False,
    collate_fn=lambda x: tuple(zip(*x))
)

print("✅ Datasets and DataLoaders ready!")


✅ Datasets and DataLoaders ready!


In [12]:
subset_images = [img for img in subset_images if os.path.exists(os.path.join(train_images_dir, img))]
print(f"✅ After removing missing files, total usable images: {len(subset_images)}")


✅ After removing missing files, total usable images: 7000


In [13]:
train_dataset = MitosisDataset(
    images_dir=train_images_dir,
    labels_dir=train_labels_dir,
    subset_images=subset_images,
    transforms=transform
)

valid_dataset = MitosisDataset(
    images_dir="/kaggle/working/mitosis-dataset-cache/valid/images",
    labels_dir="/kaggle/working/mitosis-dataset-cache/valid/labels",
    transforms=transform
)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

print("✅ Rebuilt DataLoaders successfully!")


✅ Rebuilt DataLoaders successfully!


In [14]:
import torch.optim as optim
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:
num_epochs = 20
scaler = GradScaler()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    
    for images, targets in loop:
        images = list(img.to(device) for img in images)
        targets = [{k:v.to(device) for k,v in t.items()} for t in targets]

        optimizer.zero_grad()
        with autocast():
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
        scaler.scale(losses).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += losses.item()
        loop.set_postfix(loss=running_loss/len(train_loader))

    lr_scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {running_loss/len(train_loader):.4f}")

print("🔥 Training complete!")

# Resume Training from Checkpoint

- Resuming from **epoch 15** after previously completing 14 epochs.
- Using **mixed precision** training (`GradScaler` + `autocast`) for faster GPU usage.
- Checkpoints are saved after each epoch for safe continuation.
- Monitoring **average batch loss** with tqdm progress bar.


In [ ]:
checkpoint_path = "/kaggle/working/model_epoch_14.pth"
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    start_epoch = 14
    print(f"🔁 Resuming training from epoch {start_epoch}")
else:
    start_epoch = 0
    print("🚀 Starting from scratch")

num_epochs = 20

for epoch in range(start_epoch, num_epochs):  # <-- use start_epoch here
    model.train()
    running_loss = 0.0

    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    for batch_idx, (images, targets) in enumerate(loop):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        with autocast():
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
        scaler.scale(losses).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += losses.item()
        loop.set_postfix(loss=running_loss/(batch_idx+1), refresh=True)

    lr_scheduler.step()
    torch.save(model.state_dict(), f"/kaggle/working/model_epoch_{epoch+1}.pth")
    print(f"💾 Saved checkpoint: model_epoch_{epoch+1}.pth")
    print(f"Epoch [{epoch+1}/{num_epochs}] completed, Avg Loss: {running_loss/len(train_loader):.4f}\n")


In [21]:
torch.save(model.state_dict(), "/kaggle/working/model_epoch_14.pth")
print("✅ Model saved after 14 epochs")


✅ Model saved after 14 epochs


In [22]:
model.eval()
with torch.no_grad():
    for images, targets in valid_loader:
        images = list(img.to(device) for img in images)
        outputs = model(images)
        # visualize boxes on images
        # break after a few images for quick check
        break


In [23]:
import os

checkpoint_dir = "/kaggle/working"
print("📂 Files in working directory:")
print(os.listdir(checkpoint_dir))


📂 Files in working directory:
['.virtual_documents', 'mitosis-dataset-cache', 'model_epoch_14.pth']


In [24]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model (assuming it's already defined)
model_path = "/kaggle/working/model_epoch_14.pth"
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print("✅ Model loaded successfully from:", model_path)


✅ Model loaded successfully from: /kaggle/working/model_epoch_14.pth


In [25]:
import os

checkpoint_dir = "/kaggle/working"

# List all checkpoint files
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith("model_epoch_") and f.endswith(".pth")]

if checkpoint_files:
    # Sort by epoch number
    checkpoint_files_sorted = sorted(checkpoint_files, key=lambda x: int(x.split("_")[-1].split(".")[0]))
    print("✅ Saved checkpoints:", checkpoint_files_sorted)
    
    # Latest saved epoch
    latest_epoch = int(checkpoint_files_sorted[-1].split("_")[-1].split(".")[0])
    print(f"🔹 Latest saved epoch: {latest_epoch}")
else:
    print("⚠️ No checkpoints found!")


✅ Saved checkpoints: ['model_epoch_14.pth']
🔹 Latest saved epoch: 14


In [26]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 2  # Matches your dataset (no +1 needed, FasterRCNN handles background)

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=None)

# Replace the predictor to match the number of classes
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load checkpoint
model_path = "/kaggle/working/model_epoch_14.pth"
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)

model.to(device)
model.eval()

print("✅ Model loaded successfully with num_classes =", num_classes)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 194MB/s] 


✅ Model loaded successfully with num_classes = 2


In [27]:
# Paths
test_img_dir = "/kaggle/input/mitosis-detection-dataset/test/images"
test_mask_dir = "/kaggle/input/mitosis-detection-dataset/test/labels"

# YOLO txt masks → skip; using placeholder: assume PNG or JPG masks exist
# If they’re txt, I’ll show how to convert next.
test_imgs = sorted([f for f in os.listdir(test_img_dir) if f.endswith(('.png', '.jpg'))])
test_masks = sorted([f for f in os.listdir(test_mask_dir) if f.endswith(('.png', '.jpg'))])

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [28]:
import torch.nn.functional as F

def dice_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = (target > 0.5).float()
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

def iou_score(pred, target, smooth=1e-6):
    pred = (pred > 0.5).float()
    target = (target > 0.5).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + smooth) / (union + smooth)


In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Define U-Net architecture ---
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.net(x)


class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super(UNet, self).__init__()
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(256, 512))
        self.down4 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(512, 1024))
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv1 = DoubleConv(1024, 512)
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv2 = DoubleConv(512, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv3 = DoubleConv(256, 128)
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv4 = DoubleConv(128, 64)
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5)
        x = self.conv1(torch.cat([x, x4], dim=1))
        x = self.up2(x)
        x = self.conv2(torch.cat([x, x3], dim=1))
        x = self.up3(x)
        x = self.conv3(torch.cat([x, x2], dim=1))
        x = self.up4(x)
        x = self.conv4(torch.cat([x, x1], dim=1))
        logits = self.outc(x)
        return logits


In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = UNet(n_channels=3, n_classes=1).to(device)

# Dummy validation loader simulation
# Replace with your real `valid_loader`
from torch.utils.data import DataLoader, TensorDataset
dummy_imgs = torch.randn(20, 3, 128, 128)
dummy_masks = torch.randint(0, 2, (20, 1, 128, 128)).float()
valid_loader = DataLoader(TensorDataset(dummy_imgs, dummy_masks), batch_size=4)

model.eval()

print("\n🔍 Running evaluation with progress tracking...")
predictions = []
with torch.no_grad():
    for batch_idx, (images, masks) in enumerate(tqdm(valid_loader, total=len(valid_loader))):
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs)
        predictions.append(preds.cpu())

print("\n✅ Evaluation completed!")


🔍 Running evaluation with progress tracking...


100%|██████████| 5/5 [00:00<00:00, 25.03it/s]


✅ Evaluation completed!


In [31]:
images, masks = next(iter(valid_loader))
print("Type of images:", type(images))
print("Type of masks:", type(masks))
print("Length of images:", len(images))
print("Length of masks:", len(masks))
if len(masks) > 0:
    print("Example mask element type:", type(masks[0]))
    print("Example mask keys (if dict):", masks[0].keys() if isinstance(masks[0], dict) else "Not a dict")


Type of images: <class 'torch.Tensor'>
Type of masks: <class 'torch.Tensor'>
Length of images: 4
Length of masks: 4
Example mask element type: <class 'torch.Tensor'>
Example mask keys (if dict): Not a dict


In [32]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model architecture
num_classes = 2  # 1 class (mitosis) + background
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=None)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load checkpoint
checkpoint_path = "/kaggle/working/model_epoch_14.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.to(device)
model.eval()

print("✅ Loaded trained Faster R-CNN model")


✅ Loaded trained Faster R-CNN model


In [33]:
from torchvision import transforms
from torch.utils.data import DataLoader

# Example transforms for fine-tuning
train_transform = transforms.Compose([
    transforms.Resize((512,512)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

# Use your existing MitosisDataset
train_dataset.transforms = train_transform
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Optimizer and scheduler for fine-tuning
import torch.optim as optim
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=0.002, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


YOLOv8n (nano) training codes

In [34]:
# Install YOLOv8
!pip install -q ultralytics

from ultralytics import YOLO
import os, yaml, shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00a 0:00:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [36]:
base = "/kaggle/working/mitosis-dataset-cache"

train_img = f"{base}/train/images"
train_lbl = f"{base}/train/labels"
val_img   = f"{base}/valid/images"
val_lbl   = f"{base}/valid/labels"

print(os.listdir(train_img)[:5])
print(os.listdir(train_lbl)[:5])


['A05_03Ad_jpg.rf.e01cd3554126825e1457169946b16674.jpg', 'A14_05Ca_jpg.rf.0b7c7980dcf87bf372b278edd8b3013e.jpg', 'A12_00Cb_jpg.rf.dfa931ebb25821892a113d2b581015d1.jpg', 'A17_01Ca_jpg.rf.a844fea6c676741e97253df52f158d08.jpg', 'A05_00Dd_jpg.rf.cdb6b397500830efe95f18aab995a2c6.jpg']
['A12_08Cd_jpg.rf.1a21731a230b53871ff9933e5d6ce706.txt', 'A14_05Db_jpg.rf.24e82fd7223a92be78ab19663f19e32f.txt', 'A11_01Da_jpg.rf.65eb74cffc34f21aa7f62b645cc7b1e9.txt', 'A11_06Ab_jpg.rf.9f7b84b4476768f153802a08b22ee6a6.txt', 'A11_00Bc_jpg.rf.72fae92646aa17b709b260b5835c68a0.txt']


In [37]:
model = YOLO("yolov8n.pt")
print("✅ Loaded YOLOv8n Pretrained Model")


✅ Loaded YOLOv8n Pretrained Model


In [39]:
src = "/kaggle/input/mitosis-detection-dataset"
dst = "/kaggle/working/mitosis-dataset-cache"

if not os.path.exists(dst):
    print("📦 Copying dataset...")
    shutil.copytree(src, dst)
else:
    print("⚡ Using cached dataset")

print(dst)


⚡ Using cached dataset
/kaggle/working/mitosis-dataset-cache


In [40]:
yaml_text = f"""
train: {train_img}
val: {val_img}

nc: 1
names: ["mitosis"]
"""

with open("mitosis.yaml", "w") as f:
    f.write(yaml_text)

print("✅ mitosis.yaml created.")


✅ mitosis.yaml created.


In [41]:
data_yaml = "/kaggle/working/mitosis.yaml"

yaml_content = {
    "path": dst,
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "names": ["mitosis"]
}

with open(data_yaml, "w") as f:
    yaml.dump(yaml_content, f)

print(open(data_yaml).read())


names:
- mitosis
path: /kaggle/working/mitosis-dataset-cache
test: test/images
train: train/images
val: valid/images



In [42]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")


In [43]:
import os
from ultralytics import YOLO

# -----------------------------
# 🔹 1. SAVE TRAINED MODEL
# -----------------------------
# Your trained YOLO model object = model
# Save final model manually
save_path = "/kaggle/working/yolov8_trained_final.pt"
model.save(save_path)

print(f"✅ Model saved at: {save_path}")

# -----------------------------
# 🔹 2. FIND ALL CHECKPOINTS (.pt)
# -----------------------------
checkpoint_dir = "/kaggle/working"

all_files = os.listdir(checkpoint_dir)
checkpoints = [f for f in all_files if f.endswith(".pt") or f.endswith(".pth")]

print("\n📂 Files in working directory:")
print(all_files)

print("\n🔍 Saved checkpoints:")
print(checkpoints if len(checkpoints) else "❌ No checkpoint files found")

# If you want last epoch number:
epoch_numbers = []
for ck in checkpoints:
    if "epoch" in ck:
        try:
            num = int(ck.split("epoch_")[1].split(".")[0])
            epoch_numbers.append(num)
        except:
            pass

if epoch_numbers:
    print(f"\n🔹 Latest saved epoch : {max(epoch_numbers)}")
else:
    print("\n🔹 No epoch-specific checkpoints found.")


✅ Model saved at: /kaggle/working/yolov8_trained_final.pt

📂 Files in working directory:
['yolov8_trained_final.pt', '.virtual_documents', 'yolov8n.pt', 'mitosis-dataset-cache', 'model_epoch_14.pth', 'mitosis.yaml']

🔍 Saved checkpoints:
['yolov8_trained_final.pt', 'yolov8n.pt', 'model_epoch_14.pth']

🔹 Latest saved epoch : 14


In [44]:
#Convert all labels to class 0
import os

label_dir = "/kaggle/working/mitosis-dataset-cache/train/labels"

for file in os.listdir(label_dir):
    if file.endswith(".txt"):
        path = os.path.join(label_dir, file)
        with open(path, "r") as f:
            lines = f.readlines()
        new_lines = ["0 " + " ".join(line.split()[1:]) + "\n" for line in lines]
        with open(path, "w") as f:
            f.writelines(new_lines)

print("All labels converted to class 0!")


All labels converted to class 0!


In [47]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

model.train(
    data="mitosis.yaml",
    imgsz=640,
    epochs=15,          # continue 5 more epochs
    batch=8,           # safe for GPU
    device=0,
    project="/kaggle/working",
    name="mitosis_run",
    exist_ok=True
)


Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=mitosis.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=mitosis_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

/usr/local/lib/python3.11/dist-packages/ultralytics/nn/modules/block.py:1299: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
  attn = (q.transpose(-2, -1) @ k) * self.scale
/usr/local/lib/python3.11/dist-packages/ultralytics/nn/modules/block.py:1301: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic 

train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 535.1±186.0 MB/s, size: 13.6 KB)
train: Scanning /kaggle/working/mitosis-dataset-cache/train/labels.cache... 26064 images, 15299 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 26064/26064 41.4Mit/s 0.0s
train: /kaggle/working/mitosis-dataset-cache/train/images/A03_00Ab_jpg.rf.062ac4f228af3b4c754d7532b8262949.jpg: 2 duplicate labels removed
train: /kaggle/working/mitosis-dataset-cache/train/images/A03_00Ab_jpg.rf.1200e29f13f8b60fbb5cc553eb650694.jpg: 1 duplicate labels removed
train: /kaggle/working/mitosis-dataset-cache/train/images/A03_00Ab_jpg.rf.3f08c9e5911dd07e6f1dceae369dff64.jpg: 5 duplicate labels removed
train: /kaggle/working/mitosis-dataset-cache/train/images/A03_00Ab_jpg.rf.3fa91364531025f64b1fe347545d97ee.jpg: 2 duplicate labels removed
train: /kaggle/working/mitosis-dataset-cache/train/images/A03_00Ab_jpg.rf.ac9bcef0e16f130fbf593960f860d38d.jpg: 1 duplicate labels removed
train: /kaggle/working/mitosis-dataset-cache/train/imag

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/loss.py:238: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
  pred_dist = pred_dist.view(b, a, 4, c // 4).softmax(3).matmul(self.proj.type(pred_dist.dtype))


       1/15      2.24G      1.663      4.291      1.371         14        640: 100% ━━━━━━━━━━━━ 3258/3258 10.2it/s 5:19<0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 551/551 14.4it/s 38.2s<0.1s
                   all       8807       3366     0.0634      0.368     0.0405     0.0295

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/15      2.24G      1.519      8.007      1.469          1        640: 0% ──────────── 0/3258  0.1s

...

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/loss.py:238: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:217.)
  pred_dist = pred_dist.view(b, a, 4, c // 4).softmax(3).matmul(self.proj.type(pred_dist.dtype))


      15/15      2.27G     0.7273      2.945      1.014          2        640: 100% ━━━━━━━━━━━━ 3258/3258 11.2it/s 4:52<0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 551/551 14.9it/s 37.0s<0.1s
                   all       8807       3366     0.0715      0.396     0.0558     0.0479

15 epochs completed in 1.382 hours.
Optimizer stripped from /kaggle/working/mitosis_run/weights/last.pt, 6.2MB
Optimizer stripped from /kaggle/working/mitosis_run/weights/best.pt, 6.2MB

Validating /kaggle/working/mitosis_run/weights/best.pt...
Ultralytics 8.3.228 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 551/551 15.9it/s 34.7s<0.1s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       8807       3366     0.0715      0.374     0.0648      0.056
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /kaggle/working/mitosis_run


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4132617090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/detect/train/weights/last.pt")

model.train(
    epochs=20,
    resume=True
)


In [48]:
model.save("mitosis_final.pt")


In [49]:
from ultralytics import YOLO
model = YOLO("mitosis_final.pt")


In [50]:
import os

if os.path.exists("mitosis_final.pt"):
    print("Model already trained — skipping training.")
    model = YOLO("mitosis_final.pt")
else:
    model = YOLO("yolov8n.pt")
    model.train(...)
    model.save("mitosis_final.pt")


Model already trained — skipping training.


In [52]:
import os

os.makedirs("/kaggle/working/output", exist_ok=True)
print("Output folder ready!")


Output folder ready!


In [54]:
import os

for root, dirs, files in os.walk("/kaggle/working", topdown=True):
    for f in files:
        if f == "best.pt":
            print("FOUND:", os.path.join(root, f))


FOUND: /kaggle/working/mitosis_run/weights/best.pt


In [75]:
import shutil

# Path where YOLO saved your model
src = "/kaggle/working/mitosis_run/weights/best.pt"

# Path to Kaggle output (permanent storage)
dst = "/kaggle/working/output/best.pt"

shutil.copy(src, dst)

print("Saved permanently → Check 'Output' tab for best.pt")


Saved permanently → Check 'Output' tab for best.pt
